# Goal
Paste an entire timebox json and have the agent submit it to google calendar through MCP.

## context:
we talk to the chatgpt timeboxing agent and get it to output a json in a specific format, then we paste that in here. 
that json should validate against the schema we have for timeboxes.

- prompt to be algined with how it should output the json
   - so it should know about the json and the validation we are going to be doing

In [1]:
from fateforger.agents.schedular.models import CalendarEvent

# Next steps
The functionality outlined in this notebook represents the agents capability to take a final timebox json and submit it to google calendar through MCP. So thats a single batch add tool.
After this we want to manage an entire timeboxing session from start to finish in different scenarios, so with some edge cases and flexibility to manage them.

stage 0, session management:
    confirm the start of a planning session for the day we are planning, noting the following:
    * is it a regular work day or a weekend
    * is it a holiday
    * (confirm timezone)
    q: what does this change about the following planning session? and how does it modify the next steps?

Stage 1, collecting/confirming constraints:
    * are there any pre-existing events in the calendar
    * timeboxing state (was the target day already partially planned and which anchors are already in place (wake/sleep, habits, tasks, meetings))
    Ask succinctly for: fixed events (M), commutes (C), known arrivals, unique day factors, habits scope (H), energy profile & sleep target. Confirm with “LOCKED?” before moving on. Anchoring: the timebox needs at least one fixed event, also add fixed sleep event (you can infer wakeup from end of sleep event if already in calendar, otherwise confirm)
    Format for constraints:



## notes
* we need to handle multiple calendars, work, personal, shared with partner etc

# Step 0: start planning session

for a single task we must always know:
how do we know its done?
why do you want to do it?
what is the best time to do it?
    what happens if you dont do it?
    what does it unlock when you do?
what needs to be done before it?


# step 1: Collect constraints

- fetch all event from tomorrow (and if neccesary the day after as well) through the MCP.
- fetch preferences/habits from the user profile (if any) from the Notion MCP
- fetch todos due tomorrow from ticktick MCP
- fetch any other relevant data (e.g. weather forecast, 

# step 2: create first timebox draft



* note: this process needs to work in such a way that at the end we can fetch from the calendar again, diff against it and see that there are no conflicts (so if we moved events, and then i moved that event in the calednar in the meantime, it should see that as a conflict).

In [1]:
from pydantic import BaseModel, computed_field, Field


class Rectangle(BaseModel):
    width: int = Field(alias="w")
    length: int

    @computed_field
    @property
    def area(self) -> int:
        return self.width * self.length

In [3]:
CalendarEvent.model_json_schema(by_alias=True,mode="validation")

{'$defs': {'Attendee': {'additionalProperties': False,
   'properties': {'email': {'description': 'Email address of the attendee',
     'format': 'email',
     'title': 'Email',
     'type': 'string'}},
   'required': ['email'],
   'title': 'Attendee',
   'type': 'object'},
  'ReminderOverride': {'additionalProperties': False,
   'properties': {'method': {'default': 'popup',
     'description': 'Reminder method',
     'enum': ['email', 'popup'],
     'title': 'Method',
     'type': 'string'},
    'minutes': {'description': 'Minutes before the event to trigger the reminder',
     'title': 'Minutes',
     'type': 'integer'}},
   'required': ['minutes'],
   'title': 'ReminderOverride',
   'type': 'object'},
  'Reminders': {'additionalProperties': False,
   'properties': {'useDefault': {'default': False,
     'description': 'Whether to use the default reminders',
     'title': 'Usedefault',
     'type': 'boolean'},
    'overrides': {'anyOf': [{'items': {'$ref': '#/$defs/ReminderOverride'},

In [5]:
rect = Rectangle(w=3, length=4)
rect.model_dump_json(by_alias=True)

'{"w":3,"length":4,"area":12}'

In [29]:
rect = Rectangle.model_validate_json(rect.model_dump_json(),by_name=True)